In [1]:
import pandas as pd
from time import time
from rules import rulesDict, rulesComments
from fuzzywuzzy import process,  fuzz
from collections import defaultdict
import numpy as np
import re

In [2]:
def print_pretty_table(data, cell_sep=' | ', header_separator=True):
    rows = len(data)
    cols = len(data[0])

    col_width = []
    for col in range(cols):
        columns = [data[row][col] for row in range(rows)]
        col_width.append(len(max(columns, key=len)))

    separator = "-+-".join('-' * n for n in col_width)

    for i, row in enumerate(range(rows)):
        if i == 1 and header_separator:
            print(separator)

        result = []
        for col in range(cols):
            item = data[row][col].rjust(col_width[col])
            result.append(item)

        print(cell_sep.join(result))
        

files = [
    "Electrical Products Price Sheet - 2019-2020.xlsx"
]
file = files[0]

xl = pd.ExcelFile(file)
names = xl.sheet_names

def checkFloat (n):
    return type(n) is float or (type(n).__module__ == np.__name__ 
                                and (n.dtype.type in [np.float16, np.float32, np.float64]))

In [3]:
print("Select product type:\n", "\n".join(list(map(lambda x, y: f'\t{x}: {y}', range(len(names)), names))))

page = ""
while not page.isnumeric():
    page = input("\nEnter the number: ")

productType = names[int(page)]
print("Selected:", productType)

key = process.extractOne(productType, rulesDict.keys())[0]
askList = rulesDict[key]
comList = defaultdict(str, rulesComments[key])  

Select product type:
 	0: MCCB
	1: MCB
	2: MMS
	3: MC
	4: Contactor Relay
	5: TOR
	6: EMPR

Enter the number: 2
Selected: MMS


In [6]:
dataframe = xl.parse(productType)
dataframe = dataframe.loc[:, ~dataframe.columns.str.contains('^Unnamed:')]
# Soft strip()
dataframe = dataframe.replace(r'^\s+|\s+$', '', regex=True)
answers = defaultdict(str)

for parametr in askList:

    characterValue = ""
    for columnName in dataframe:
        if fuzz.ratio(columnName.replace('\n', ""), parametr) > 90:
            tempDataSeries = dataframe[columnName].astype(np.str_)
            break

    if comList and comList[parametr]:
        print(comList[parametr])
    variants = np.unique(tempDataSeries)
    variantslen = len(variants)
    variants.sort()
    if variantslen == 1:
        characterValue = variants[0]
    else:
        print('-' * 80, f'\nEnter value for "{parametr}"')
        if variantslen <= 10:
            print('Variants:\n', "\n".join(list(map(lambda x, y: f'\t{x}: {y}', range(len(variants)), variants))))
            while not characterValue:
                answerInp = input('Enter correct number: ')
                answer = answerInp
                if answer.isnumeric():
                    answer = int(answer)
                    if -1 < answer < variantslen:
                        characterValue = variants[answer]
        else:
#             variants = variants.astype(np.str_)
            print('Examples:\n\t', ", ".join(variants[:10]))
            while not characterValue:
                answerInp = input('Enter value: ')
                answer = answerInp
                if "rated current" in parametr.lower():
                    answer = float((re.sub(r"[^.0-9]", "", answer.replace(",", "."))))
                    floatVariants = list(map(lambda z: (float(re.sub(r"[^.0-9]", "", z)), z), variants))
                    floatVariants.sort(key=lambda x: x[0])
                    for sortedVarianValue in floatVariants:
                        if answer <= sortedVarianValue[0]:
                            characterValue = sortedVarianValue[1]
                            break
                    if not characterValue: print(f"Maximum: {max(floatVariants)[1]}")
                else:
                    fastKey = lambda xxx: max(xxx, key=lambda x: x[1])[0]

                    selected1 = process.extractOne(answer.replace(" ", ""), variants)[0]

                    selected2 = fastKey(list(map(lambda x: (x, fuzz.ratio(answer, x)), variants)))

                    temp = list(
                        zip(*list(map(lambda z: list(map(lambda x: fuzz.ratio(z, x), variants)), answer.split()))))
                    selected3 = fastKey(list(map(lambda x, z: (z, sum(x)), temp, variants)))

                    temp = list(
                        zip(*list(map(lambda z: list(map(lambda x: fuzz.WRatio(z, x), variants)), answer.split()))))
                    selected4 = fastKey(list(map(lambda x, z: (z, sum(x)), temp, variants)))

                    if not (selected1 == selected2 == selected3 == selected4):
                        selected = [selected1, selected2, selected3, selected4]
                        c = Counter(selected)
                        for elem in c:
                            if c[elem] == 3:
                                if (input(f'Do you mean: {elem}? (y - continue, other key - repeat)\n').lower()=='y'):
                                    characterValue = elem
                        if not characterValue:
                            print("Сan't determine the appropriate value. Here's a similar one:\n\t", selected)
                            print("Please repeat")
                    else:
                        characterValue = selected1
            answers[columnName] = answerInp
    dataframe = dataframe[tempDataSeries == np.array(characterValue, tempDataSeries.dtype)]
    if dataframe.shape[0] == 1: break

-------------------------------------------------------------------------------- 
Enter value for "Rated current"
Examples:
	 0.16A, 0.25A, 0.4A, 0.63A, 1.6A, 100A, 10A, 13A, 17A, 1A
Enter value: 11
-------------------------------------------------------------------------------- 
Enter value for "Breaking Capacity"
Variants:
 	0: H
	1: HI
	2: S
Enter correct number: 1
-------------------------------------------------------------------------------- 
Enter value for "Frame size"
Variants:
 	0: 32
	1: 63
Enter correct number: 0


In [7]:
if dataframe.shape[0] == 1:
    text = "Exact match found"
else:
    text = f'{dataframe.shape[0]} items found'
print("\n{:-^80}\n".format(text))

for index in dataframe.index:
    itemDesc = [["Property", "Value", "You asked"]]
    printDiff = False
    row = dataframe.loc[index]
    for i in dataframe:
        if pd.notnull(row[i]):
            diff = fuzz.partial_ratio(str(row[i]), answers[i]) < 90 and answers[i] != ""
            printDiff |= diff
            itemDesc.append(list(map(
                lambda x: (str(x) if not checkFloat(x) else f'{x:.2f}').replace("\n", ""),
                [i, row[i], answers[i] if diff else ""]
            )))

    if not printDiff:
        itemDesc = list(zip(*itemDesc))
        itemDesc.pop()
        itemDesc = list(zip(*itemDesc))

    print_pretty_table(itemDesc)
    print("\n")


-------------------------------Exact match found--------------------------------

               Property |            Value | You asked
------------------------+------------------+----------
                   Item |        705002800 |          
            Description | MMS-32HI 13A EXP |          
   Series Name(Class 2) |         MMS-32HI |          
             Frame size |               32 |          
       BreakingCapacity |               HI |          
      InstantaneousType |                I |          
          Rated current |              13A |        11
Adjustablecurrent range |                - |          
                 Suffix |              EXP |          
             List price |           166.47 |          


